In [1]:
import pandas as pd
import json
import requests
import time
import csv
import os

os.chdir('/Users/samiadam/Repositories/Projects/FPL_Project/')
os.getcwd()

'/Users/samiadam/Repositories/Projects/FPL_Project'

In [8]:
df20 = pd.read_csv("./data/2020-21/merged_gw.csv")
df24 = pd.read_csv("./data/2024-25/merged_gw.csv")

[col for col in df24.columns if col not in df20.columns]

['expected_assists',
 'expected_goal_involvements',
 'expected_goals',
 'expected_goals_conceded',
 'starts']

## FPL Data

### Get Functions

In [2]:
def get_data():
    url = "https://fantasy.premierleague.com/api/bootstrap-static/"
    response = ""
    
    while response == "":
        try:
            response = requests.get(url)
        except:
            time.sleep(5)
    
    if response.status_code != 200:
        raise Exception(f"Response Code: {response.status_code}")
    
    data = json.loads(response.content)
    return data

def get_fixture_data():
    url = "https://fantasy.premierleague.com/api/fixtures/"

    response = ""
    while response == "":
        try:
            response = requests.get(url)
        except:
            time.sleep(5)

    if response.status_code != 200:
        raise Exception(f"Response Code: {response.status_code}")

    data = json.loads(response.text)
    return data

def get_individual_data(id):
    base_url = "https://fantasy.premierleague.com/api/element-summary/"
    url = base_url + str(id) + "/"
    response = ""
    
    while response == "":
        try:
            response = requests.get(url)
        except:
            time.sleep(5)
            
    if response.status_code != 200:
        raise Exception(f"Response Code: {response.status_code}")
    
    data = json.loads(response.text)
    return data
    


### Parsing Functions

In [3]:
def extract_stat_names(stat_dict):
    stat_names = []
    for key in stat_dict.keys():
        stat_names += [key]

    return stat_names

def parse_players(list_of_players, base_path):
    stat_names = extract_stat_names(list_of_players[0])

    file_name = os.path.join(base_path,  "players_raw.csv")
    os.makedirs(os.path.dirname(base_path), exist_ok=True)
    
    f = open(file_name, "w+", encoding= "utf8", newline= "")
    w = csv.DictWriter(f, sorted(stat_names))

    w.writeheader()
    for player in list_of_players:
        w.writerow({k:str(v).encode('utf-8').decode('utf-8') for k, v in player.items()})
        
def parse_fixtures(data, base_path):
    fixtures_df = pd.DataFrame.from_records(data)
    fixtures_df.to_csv(os.path.join(base_path, "fixtures.csv"), index= False)
    
def parse_team_data(data, base_path):
    teams_df = pd.DataFrame.from_records(data)
    teams_df.to_csv(os.path.join(base_path, "teams.csv"), index= False)

def parse_player_gw_history(gw_history_list, base_path, name, id):
    if gw_history_list:
        stat_names = extract_stat_names(gw_history_list[0])
        file_path = os.path.join(base_path, f"{name}_{id}", "gw.csv")
        os.makedirs(os.path.dirname(file_path), exist_ok= True)

        with open(file_path, "w+", encoding= "utf-8", newline= "") as file:
            w = csv.DictWriter(file, sorted(stat_names))
            w.writeheader()
            w.writerows(gw_history_list)
            
def parse_player_season_history(player_hist_list, base_path, name, id):
    if player_hist_list:
        stat_names = extract_stat_names(player_hist_list[0])
        file_path = os.path.join(base_path, f"{name}_{id}", "history.csv")
        os.makedirs(os.path.dirname(file_path), exist_ok= True)
        
        with open(file_path, "w+", encoding= "utf-8", newline= "") as file:
            w = csv.DictWriter(file, sorted(stat_names))
            w.writeheader()
            w.writerows(player_hist_list)

### Cleaning Functions

In [4]:
def clean_players(file_name, base_path):    
    headers = ['first_name', 'second_name', 'goals_scored', 'assists', 'total_points', 'minutes', 'goals_conceded', 'creativity', 'influence', 'threat', 'bonus', 'bps', 'ict_index', 'clean_sheets', 'red_cards', 'yellow_cards', 'selected_by_percent', 'now_cost', 'element_type']

    raw = open(base_path + file_name, "r+", encoding= "utf-8")
    out_name = base_path + "players_clean.csv"
    clean = open(out_name, "w+", encoding= "utf-8", newline = "")

    reader = csv.DictReader(raw)
    writer = csv.DictWriter(clean, headers, extrasaction= "ignore")
    writer.writeheader()
    for line in reader:
        if line['element_type'] == '1':
            line['element_type'] = 'GK'
        elif line['element_type'] == '2':
            line['element_type'] = 'DEF'
        elif line['element_type'] == '3':
            line['element_type'] = 'MID'
        elif line['element_type'] == '4':
            line['element_type'] = 'FWD'
        else:
            print("Oh boy")
        writer.writerow(line)
        
def id_players(file_name, base_path):
    headers = ["first_name", "second_name", "id"]
    raw = open(base_path + file_name, "r+", encoding= "utf-8")
    clean_name = base_path + "player_idlist.csv"
    os.makedirs(os.path.dirname(clean_name), exist_ok=True)

    clean = open(clean_name, "w+", encoding= "utf-8", newline= "")
    reader = csv.DictReader(raw)
    writer = csv.DictWriter(clean, headers, extrasaction= "ignore")

    writer.writeheader()
    for line in reader:
        writer.writerow(line)
        
def get_player_ids(base_path):
    file_name = base_path + "player_idlist.csv"

    file = open(file_name, "r+", encoding= "utf-8")
    reader = csv.DictReader(file)
    player_ids = {}

    for line in reader:
        id = line["id"]
        name = line["first_name"] + "_" + line["second_name"].replace(" ", "_")
        player_ids[id] = name
        
    return player_ids

In [5]:
season = "2024-25"
base_path = "/Users/samiadam/Repositories/Projects/FPL_Project/data/" + season + "/"

### Collectors

In [6]:
def get_teams(directory):
    teams = {}
    path = directory + "/teams.csv"
    
    with open(path, "r") as file_in:
        reader = csv.DictReader(file_in)
        for row in reader :
            teams[int(row["id"])] = row["name"]
    
    return teams

def get_fixtures(directory):
    fix_home = {}
    fix_away = {}
    
    with open(os.path.join(directory, "fixtures.csv"), "r") as file_in:
        reader = csv.DictReader(file_in)
        for row in reader:
            fix_home[int(row['id'])] = int(row['team_h'])
            fix_away[int(row['id'])] = int(row['team_a'])
            
    return fix_home, fix_away

def get_positions(directory):
    positions = {}
    names = {}
    pos_dict = {'1': "GK", '2': "DEF", '3': "MID", '4': "FWD"}
    
    with open(directory + "/players_raw.csv", 'r',encoding="utf-8") as file_in:
        reader = csv.DictReader(file_in)
        for row in reader:
            positions[int(row['id'])] = pos_dict[row['element_type']] 
            names[int(row['id'])] = row['first_name'] + ' ' + row['second_name']
    
    return names, positions

def collect_gw(base_directory, output_dir, gw):
    rows = []
    fieldnames = []

    fix_home, fix_away = get_fixtures(base_directory)
    teams = get_teams(base_directory)
    player_names, player_positions = get_positions(base_directory)

    for root, dirs, files in os.walk(base_directory + "players/"):
        for file in files:
            if file == "gw.csv":
                path = os.path.join(root, file)
                
                with open(path, "r") as file_in:
                    reader = csv.DictReader(file_in)
                    fieldnames = reader.fieldnames

                    for row in reader:
                        if int(row['round']) == gw:
                            id = int(os.path.basename(root).split('_')[-1])
                            name = player_names[id]
                            position = player_positions[id]
                            fixture = int(row['fixture'])
                            if row['was_home'] == True or row['was_home'] == "True":
                                row['team'] = teams[fix_home[fixture]]
                            else:
                                row['team'] = teams[fix_away[fixture]]
                            row['name'] = name
                            row['position'] = position
                            rows += [row]

    fieldnames = ['name', 'position', 'team', 'xP'] + fieldnames

    os.makedirs(output_dir, exist_ok= True)
    with open(os.path.join(output_dir, "gw_" + str(gw) + ".csv"), "w+") as out_file:
        w = csv.DictWriter(out_file, fieldnames= fieldnames)
        w.writeheader()
        w.writerows(rows)
        
def collect_all_gws(base_directory, output_dir, current_gw):
    for gw in range(1, current_gw):
        collect_gw(base_directory, output_dir, gw)



### Others

In [7]:
season = "2024-25"
base_path = "/data/" + season + "/"


def global_scraper(season: str):
    print("Scraping Data \n")
    overall_start_time = time.time()
    
    # Setting vars
    base_path = "./data/" + season + "/"
    
    # Getting Data
    print("Getting data...", end= " ")
    start = time.time()
    
    data = get_data()
    end = time.time()
    
    elapsed_time = end - start
    print(f"DONE ({elapsed_time:.2f} seconds)")

    # Parsing Data
    print("Parsing Data...", end= " ")
    start = time.time()
    parse_players(data["elements"], base_path)
        
    gw_num = 0
    events = data["events"]
    for event in events:
        if event["is_current"] == True:
            gw_num = event["id"]
            break
    
    end = time.time()
    elapsed_time = end - start
    print(f"DONE ({elapsed_time:.2f} seconds)")
            
    # Cleaning Data
    print("Cleaning Summary Data...", end= " ")
    start = time.time()
    
    clean_players("players_raw.csv", base_path)
    
    end = time.time()
    elapsed_time = end - start
    print(f"DONE ({elapsed_time:.2f} seconds)")

    # Fixtures
    print("Getting Fixtures Data...", end= " ")
    start = time.time()
    
    fixture_data = get_fixture_data()
    parse_fixtures(fixture_data, base_path)
    
    end = time.time()
    elapsed_time = end - start
    print(f"DONE ({elapsed_time:.2f} seconds)")

    # Team Data
    print("Getting Team Data...", end= " ")
    start = time.time()
    
    team_data = data["teams"]
    parse_team_data(team_data, base_path)

    end = time.time()
    elapsed_time = end - start
    print(f"DONE ({elapsed_time:.2f} seconds)")

    # Player Data
    print("Getting Player Data...", end= " ")
    start = time.time()
    
    id_players("players_raw.csv", base_path)
    player_ids = get_player_ids(base_path)

    player_base_path = base_path + 'players/'
    gw_base_path = base_path + 'gws/'

    # num_players = len(data["elements"]) idk if needed
    
    # Player Individual Data
    for id, name in player_ids.items():
        player_data = get_individual_data(id)
        parse_player_season_history(player_data["history_past"], player_base_path, name, id)
        parse_player_gw_history(player_data["history"], player_base_path, name, id)

    end = time.time()
    elapsed_time = end - start
    print(f"DONE ({elapsed_time:.2f} seconds)")

    # Gameweek Data
    if gw_num > 0:
        print("Getting GW Data...", end= "")
        start = time.time()
        
        collect_gw(base_path, gw_base_path, gw_num)
        
        end = time.time()
        elapsed_time = end - start
        print(f"DONE ({elapsed_time:.2f} seconds)")
        
        print("Merging GW Data... ", end= " ")
        start = time.time()
        
        collect_all_gws(base_path, gw_base_path, gw_num)
        
        end = time.time()
        elapsed_time = end - start
        print(f"DONE ({elapsed_time:.2f} seconds)")
        
    overall_elapsed_time = time.time() - overall_start_time
    print(f"\nGlobal scraper completed in {overall_elapsed_time:.2f} seconds.")


## FBREF Data

In [8]:
from requests.exceptions import HTTPError
import requests
import time

def get_individual_player_data(fbref_id, fbref_name):
    fbref_name_link = fbref_name.replace(" ", "-")
    link = f"https://fbref.com/en/players/" + fbref_id + "/matchlogs/2024-2025/" +  fbref_name_link + "-Match-Logs"
    
    try:
        
        print(f"Fetching data for {fbref_name}")
        
        response = requests.get(link)
        print(response.text)
        response.raise_for_status()
        all_performances_df = pd.read_html(response.content, attrs= {"id": "matchlogs_all"})[0]
        all_performances_df.columns = [' '.join(col).strip() if not col[0].startswith("Unnamed") else col[1] for col in all_performances_df.columns.values]
        
        prem_performances_df = all_performances_df[all_performances_df["Comp"] == "Premier League"].copy()
        prem_performances_df.loc[:, "name"] = fbref_name
        
    except HTTPError as e:
        if response.status_code == 429:
            wait_time = int(response.headers.get("Retry-After", 10))
            print(f"Status Code: 429. Retrying in {wait_time} seconds...")
            
            time.sleep(wait_time)
            attempt +=1
            
        else:
            raise e
    
    except Exception as e:
        raise e
    
    return prem_performances_df

    
def collect_players_data(df, call_rate= 9.5):
    if call_rate > 10:
        raise Exception("Call rate too high. It must be below 10.")
    
    wait_time = 60/call_rate
    
    df_list = []
    for idx, row in df.head(1).iterrows():
        fbref_id = row["id_fbref"]
        fbref_name = row["name_fbref"]
        
        player_df = get_individual_player_data(fbref_id, fbref_name)
        df_list.append(player_df)
        
        time.sleep(wait_time)
    
    return df_list

## Merging IDs

In [9]:
import json
import pandas as pd
from fuzzywuzzy import process
import unicodedata
import numpy as np
import time


def remove_special_letters(text):
    normalized_text = unicodedata.normalize('NFKD', text)
    return ''.join([c for c in normalized_text if not unicodedata.combining(c)])

def fuzzy_match(name, choices, threshold, scorer=process.extractOne):
    match, score = scorer(name, choices)
    return match if score >= threshold else np.NaN

def sorted_fuzzy_match(name, choices, scorer=process.extract):
    matches = scorer(name, choices)
    return sorted(matches, key=lambda x: x[1], reverse=True)

def load_dfs(fbref_id_path, fpl_id_path):
    # Load the files
    fbref_ids = pd.read_csv(fbref_id_path)
    fpl_ids = pd.read_csv(fpl_id_path)

    fpl_ids['full_name_abbr'] = fpl_ids['first_name'].str.split(" ").str[0] + " " + fpl_ids["second_name"].str.split(" ").str[-1]
    fpl_ids['full_name_full'] = fpl_ids["first_name"] + " " + fpl_ids["second_name"]
    fbref_ids[['first_name', 'second_name']] = fbref_ids['name'].str.split(' ',n= 1, expand=True)
    
    return fbref_ids, fpl_ids

def check_name_match(df, fbref_df):
    
    matched_indices = []

    for idx, row in df[~df["fuzzy_match"].isna()].iterrows():
        full_name = row["full_name_full"]
        match = row["fuzzy_match"]
        print(f"{full_name}: {match}")
        agree = input("Do they match? (Press Enter for yes):\n")
        
        if agree.lower() == "": 
            matching_ids = fbref_df[fbref_df["name"] == match]["id"]
            if not matching_ids.empty:
                df.at[idx, "id_fbref"] = matching_ids.iloc[0]
                matched_indices.append(idx)
        else:
            df.at[idx, "fuzzy_match"] = np.NaN

def map_name_match(df, fbref_df):
    
    matched_indices = []

    for idx, row in df[~df["fuzzy_match"].isna()].iterrows():
        match = row["fuzzy_match"]
        
        matching_ids = fbref_df[fbref_df["name"] == match]["id"]
        if not matching_ids.empty:
            df.at[idx, "id_fbref"] = matching_ids.iloc[0]
            matched_indices.append(idx)

    df.update(df.loc[matched_indices])
    return df

def sift_names(df, fbref_df, level, name):
    print(f"\n{level.title()} Sift...")
    matched_indices = []
    missing_df = df[df["fuzzy_match"].isna()]
    
    for i, (idx, row) in enumerate(missing_df.iterrows(), 1):
        fname = row['first_name']
        lname = row['second_name']
        
        print(f"Debug - name parameter: {name}")
        print(f"Debug - fname: {fname}, lname: {lname}")
        
        if name == "first":
            filter_name = remove_special_letters(fname)
        elif name == "last":            
            filter_name = remove_special_letters(lname.split(" ")[-1])
        
        print(f"Debug - filter_name: {filter_name}")
        matching_fnames = fbref_df[fbref_df["name"].str.contains(filter_name, case=False, na=False)]["name"].tolist()

        if matching_fnames:
            if level == "strict":
                related_names = sorted_fuzzy_match(fname if name == "first" else lname, matching_fnames)
                related_names = [name for name, score in related_names]
            if level == "loose":
                related_names = matching_fnames

            print(f"\n({i}/{missing_df.shape[0]}) Matching for: {fname} {lname}")
            
            for i, (related_name) in enumerate(related_names, 1):
                print(f"{i}. {related_name}")
            
            print("0. None of the above")
            
            choice = input("Enter the number of the correct match (0 for none): ")
            
            if choice.isdigit():
                choice = int(choice)
                if 1 <= choice <= len(related_names):
                    selected_name = related_names[choice - 1][0]
                    df.at[idx, "fuzzy_match"] = selected_name
                    matching_id = fbref_df[fbref_df["name"] == selected_name]["id"].iloc[0]
                    df.at[idx, "id_fbref"] = matching_id
                    matched_indices.append(idx)
                    print(f"Selected: {selected_name}")
                
                elif choice == 0:
                    print("No match selected.")
                
                else:
                    print("Invalid choice. No match selected.")
            else:
                print("Invalid input. No match selected.")
        else:
            print(f"No matches found for {fname} {lname}")

    df.update(df.loc[matched_indices])
    
    return df

## Testing Seasonality

In [10]:
teams24 = pd.read_csv("./data/2024-25/teams.csv")
teams23 = pd.read_csv("./data/2023-24/teams.csv")
teams22 = pd.read_csv("./data/2022-23/teams.csv")
teams21 = pd.read_csv("./data/2021-22/teams.csv")
teams20 = pd.read_csv("./data/2020-21/teams.csv")

clubs_link_dict = {
"Liverpool": "https://fbref.com/en/squads/822bd0ba/2024-2025/all_comps/Liverpool-Stats-All-Competitions",
"Man City": "https://fbref.com/en/squads/b8fd03ef/2024-2025/all_comps/Manchester-City-Stats-All-Competitions",
"Arsenal": "https://fbref.com/en/squads/18bb7c10/2024-2025/all_comps/Arsenal-Stats-All-Competitions",
"Chelsea": "https://fbref.com/en/squads/cff3d9bb/2024-2025/all_comps/Chelsea-Stats-All-Competitions",
"Aston Villa": "https://fbref.com/en/squads/8602292d/2024-2025/all_comps/Aston-Villa-Stats-All-Competitions",
"Brighton": "https://fbref.com/en/squads/d07537b9/2024-2025/all_comps/Brighton-and-Hove-Albion-Stats-All-Competitions",
"Newcastle": "https://fbref.com/en/squads/b2b47a98/2024-2025/all_comps/Newcastle-United-Stats-All-Competitions",
"Fulham": "https://fbref.com/en/squads/fd962109/2024-2025/all_comps/Fulham-Stats-All-Competitions",
"Spurs": "https://fbref.com/en/squads/361ca564/2024-2025/all_comps/Tottenham-Hotspur-Stats-All-Competitions",
"Nott'm Forest": "https://fbref.com/en/squads/e4a775cb/2024-2025/all_comps/Nottingham-Forest-Stats-All-Competitions",
"Brentford": "https://fbref.com/en/squads/cd051869/2024-2025/all_comps/Brentford-Stats-All-Competitions",
"West Ham": "https://fbref.com/en/squads/7c21e445/2024-2025/all_comps/West-Ham-United-Stats-All-Competitions",
"Bournemouth": "https://fbref.com/en/squads/4ba7cbea/2024-2025/all_comps/Bournemouth-Stats-All-Competitions",
"Man Utd": "https://fbref.com/en/squads/19538871/2024-2025/all_comps/Manchester-United-Stats-All-Competitions",
"Leicester": "https://fbref.com/en/squads/a2d435b3/2024-2025/all_comps/Leicester-City-Stats-All-Competitions",
"Everton": "https://fbref.com/en/squads/d3fd31cc/2024-2025/all_comps/Everton-Stats-All-Competitions",
"Ipswich": "https://fbref.com/en/squads/b74092de/2024-2025/all_comps/Ipswich-Town-Stats-All-Competitions",
"Crystal Palace": "https://fbref.com/en/squads/47c64c55/2024-2025/all_comps/Crystal-Palace-Stats-All-Competitions",
"Southampton": "https://fbref.com/en/squads/33c895d4/2024-2025/all_comps/Southampton-Stats-All-Competitions",
"Wolves": "https://fbref.com/en/squads/8cec06e1/2024-2025/all_comps/Wolverhampton-Wanderers-Stats-All-Competitions",
"Burnley": "https://fbref.com/en/squads/943e8050/2024-2025/all_comps/Burnley-Stats-All-Competitions",
"Luton": "https://fbref.com/en/squads/e297cd13/2024-2025/all_comps/Luton-Town-Stats-All-Competitions",
"Sheffield Utd": "https://fbref.com/en/squads/1df6b87e/2024-2025/all_comps/Sheffield-United-Stats-All-Competitions",
"Leeds": "https://fbref.com/en/squads/5bfb9659/2024-2025/all_comps/Leeds-United-Stats-All-Competitions",
"Norwich": "https://fbref.com/en/squads/1c781004/2024-2025/all_comps/Norwich-City-Stats-All-Competitions",
"Watford": "https://fbref.com/en/squads/2abfe087/2024-2025/all_comps/Watford-Stats-All-Competitions",
"West Brom":"https://fbref.com/en/squads/60c6b05f/2024-2025/all_comps/West-Bromwich-Albion-Stats-All-Competitions"
}

In [11]:
team_list24 = teams24["name"].to_list()
team_list23 = teams23["name"].to_list()
team_list22 = teams22["name"].to_list()
team_list21 = teams21["name"].to_list()
team_list20 = teams20["name"].to_list()

master_team_list = set(team_list24 + team_list23 + team_list22 + team_list21 + team_list20)
[x for x in master_team_list if not x in clubs_link_dict.keys()]

[]

## Merging Previous fuzzy match

In [12]:
# Define file paths
fbref_path = "data/2024-25/fbref_ids.csv"
fpl_path = 'data/2024-25/player_idlist.csv'

# Load and preprocess data
fbref_ids, fpl_ids = load_dfs(fbref_path, fpl_path)

# Merge datasets
merged = pd.merge(
    fpl_ids, 
    fbref_ids, 
    "left", 
    on=["first_name", "second_name"], 
    suffixes=["_fpl", "_fbref"]
)


# Get the previous season's data
prev_season = "2023-24"
previous_merged_ids = pd.read_csv(f"data/{prev_season}/player_compiled_ids.csv")

# Merge the current data with the previous season based on first_name and second_name
merged_with_prev = pd.merge(
    merged, 
    previous_merged_ids[["first_name_fpl", "second_name_fpl", "id_fbref"]], 
    how="left", 
    left_on=["first_name", "second_name"], 
    right_on=["first_name_fpl", "second_name_fpl"], 
    suffixes=("", "_prev")
)

# Update the id_fbref in the merged dataframe if it is missing (i.e., NaN) in the current season
merged_with_prev["id_fbref"] = merged_with_prev["id_fbref"].combine_first(merged_with_prev["id_fbref_prev"])

# Drop the temporary columns used for merging
merged_with_prev = merged_with_prev.drop(columns=["first_name_fpl", "second_name_fpl", "id_fbref_prev"])

# Overwrite the merged dataframe with the updated values
merged = merged_with_prev

## Merging FBref IDs with previous season

In [13]:
def get_previous_season(season_str: str) -> str:
    # Split the season string into start and end years
    start_year = int(season_str[:4])
    end_year = int(season_str[5:])
    
    # Subtract 1 from both years
    previous_start_year = start_year - 1
    previous_end_year = end_year - 1
    
    # Format the previous season in "YYYY-YY" format
    return f"{previous_start_year}-{str(previous_end_year)[-2:]}"

In [14]:
season = "2024-25"

# Define file paths
fbref_path = f"./data/{season}/fbref_ids.csv"
fpl_path = f'./data/{season}/player_idlist.csv'

# Load and preprocess data
fbref_ids, fpl_ids = load_dfs(fbref_path, fpl_path)

# Merge datasets
merged = pd.merge(
    fpl_ids, 
    fbref_ids, 
    "left", 
    on=["first_name", "second_name"], 
    suffixes=["_fpl", "_fbref"]
)


    # Get the previous season's data
prev_season = get_previous_season(season)
previous_merged_ids = pd.read_csv(f"./data/{prev_season}/player_compiled_ids.csv")

# Merge the current data with the previous season based on first_name and second_name
merged_with_prev = pd.merge(
    merged, 
    previous_merged_ids[["first_name_fpl", "second_name_fpl", "id_fbref"]], 
    how="left", 
    left_on=["first_name", "second_name"], 
    right_on=["first_name_fpl", "second_name_fpl"], 
    suffixes=("", "_prev")
)

# Update the id_fbref in the merged dataframe if it is missing (i.e., NaN) in the current season
merged_with_prev["id_fbref"] = merged_with_prev["id_fbref"].combine_first(merged_with_prev["id_fbref_prev"])

# Drop the temporary columns used for merging
merged_with_prev = merged_with_prev.drop(columns=["first_name_fpl", "second_name_fpl", "id_fbref_prev"])

# Overwrite the merged dataframe with the updated values
merged = merged_with_prev

## Data to Send DB

In [57]:
season = "2024-25"
fbref_merged_gw = pd.read_csv(f"./data/{season}/fbref_merged_gw_data.csv")

fbref_merged_gw["Start"] = fbref_merged_gw["Start"].replace("Y*", "Y")
fbref_merged_gw["Captain"]

0       Y
1       Y
2       Y
3       Y
4       Y
       ..
2989    Y
2990    Y
2991    Y
2992    Y
2993    Y
Name: Captain, Length: 2994, dtype: object

In [ ]:
start_map = {"Y": 1, "N": 0}
fbref_merged_gw["Start"] = fbref_merged_gw["Start"].replace("Y*", "Y")
fbref_merged_gw["Start"] = fbref_merged_gw["Start"].map(start_map)

fbref_merged_gw.to_csv()

,Date,Day,Comp,Round,Venue,Result,Squad,Opponent,Start,Pos,...,Performance Fls,Performance Fld,Performance Off,Performance Crs,Performance TklW,Performance OG,Performance PKwon,Performance PKcon,Season,Captain
0,2024-08-17,Sat,Premier League,1,Home,W 2–0,Arsenal,Wolves,NaN,"FW,LM",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-25,Y
1,2024-08-24,Sat,Premier League,2,Away,W 2–0,Arsenal,Aston Villa,NaN,FW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-25,Y
2,2024-08-31,Sat,Premier League,3,Home,D 1–1,Arsenal,Brighton,NaN,FW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-25,Y
3,2024-09-15,Sun,Premier League,4,Away,W 1–0,Arsenal,Tottenham,NaN,"FW,LM",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-25,Y
4,2024-09-22,Sun,Premier League,5,Away,D 2–2,Arsenal,Manchester City,NaN,"RM,FW",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-25,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2989,2024-11-03,Sun,Premier League,10,Home,D 1–1,Manchester Utd,Chelsea,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-25,Y
2990,2024-10-26,Sat,Premier League,9,Home,D 2–2,Brighton,Wolves,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-25,Y
2991,2024-10-27,Sun,Premier League,9,Away,D 2–2,Liverpool,Arsenal,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-25,Y
2992,2024-10-27,Sun,Premier League,9,Away,L 1–2,Manchester Utd,West Ham,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-25,Y
